In [ ]:
# !pip install autopep8

In [ ]:
# !wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec_small.pt

In [ ]:
# !wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec_vox_new.pt
# !wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/libri960_big.pt

In [ ]:
%ls /home/seld_wav2vec2/src/seld_wav2vec2

In [ ]:
%ls

In [ ]:
import torch
import fairseq
from pathlib import Path
from argparse import Namespace
from collections import namedtuple

user_dir = "/home/seld_wav2vec2/src/seld_wav2vec2"
Arg = namedtuple("Arg", ["user_dir"])
arg = Arg(user_dir.__str__())
fairseq.utils.import_user_module(arg)

cp = "wav2vec_small.pt"
# cp = "wav2vec_vox_new.pt"
# cp = "libri960_big.pt"

model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp], strict=True)
model = model[0]
model.eval()

In [ ]:
# cfg

cfg.model

In [ ]:
w2v_state = model.state_dict()

In [ ]:
print(w2v_state.keys())

In [ ]:
w2v_state['feature_extractor.conv_layers.1.0.weight']

In [ ]:
from omegaconf.dictconfig import DictConfig
import copy

print(type(cfg.model))

content = dict(cfg.model)
print(content)

res = DictConfig(content = {**content, **{'in_channels': 4, 'in_conv_groups': 1}})
             
res._name = 'wav2vec2_ch'

res

In [ ]:
# model_spec = task.build_model(res, from_checkpoint=False)

model_spec = task.build_model(res, from_checkpoint=True)

In [ ]:
model_spec

In [ ]:
del model

state = fairseq.checkpoint_utils.load_checkpoint_to_cpu(cp)

In [ ]:
state_dict = model_spec.state_dict()
state_model = state["model"].copy()
for key in state["model"]:
    if key in state_dict.keys():
        if state["model"][key].shape != state_dict[key].shape:
            print("key {} is not matching".format(key))
            state_model.pop(key)

model_spec.load_state_dict(state_model, strict=False) 

In [ ]:
print(model_spec.cfg)

print(type(model_spec.cfg))

# save model

In [ ]:
print(state.keys())

# print(state['model'].keys())

state['args'].arch = 'wav2vec2_ch'

print(state['args'].arch)

print(state['args'])

print(state['cfg'].keys())

In [ ]:
print(state['cfg'])

In [ ]:
# state['cfg']['task'].normalize = True 

print(state['cfg']['task'])

In [ ]:
state['model'] = model_spec.state_dict()

In [ ]:
state['cfg']['model'] = res

In [ ]:
print(state['cfg']['model'])

In [ ]:
state

In [ ]:
cp_save = "w2v_audio_base_4ch_unorm.pt"
# cp_save = "w2v_audio_base_4ch_unorm_mask.pt"
# cp_save = "w2v_audio_large_4ch_norm.pt"
# cp_save = "w2v_audio_large_4ch_unorm.pt"

!rm -rf {cp_save}

torch.save(state, cp_save)

In [ ]:
del model_spec, state, state_model

import gc

gc.collect()

# Load 

In [ ]:
# import torch
# import fairseq
# from pathlib import Path
# from argparse import Namespace
# from collections import namedtuple

# user_dir = Path("/content/sound_events_w2v_d2v/src/sound_events_w2v_d2v/model")
# Arg = namedtuple("Arg", ["user_dir"])
# arg = Arg(user_dir.__str__())
# fairseq.utils.import_user_module(arg)

In [ ]:
# cp_save = "w2v_audio_large_4ch_norm.pt"

state_new = fairseq.checkpoint_utils.load_checkpoint_to_cpu(cp_save)

In [ ]:
print(state_new.keys())

print(state_new['model'].keys())

In [ ]:
print(state_new['cfg'].keys())

print(state_new['cfg']['model'])

In [ ]:
del state_new

gc.collect()

In [ ]:
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_save], strict=True)
model = model[0]
model.eval()

In [ ]:
model

In [ ]:
saved_4ch_w2v_state = model.state_dict()

In [ ]:

assert torch.equal(w2v_state['feature_extractor.conv_layers.1.0.weight'], saved_4ch_w2v_state['feature_extractor.conv_layers.1.0.weight'])
assert torch.equal(w2v_state['feature_extractor.conv_layers.2.0.weight'], saved_4ch_w2v_state['feature_extractor.conv_layers.2.0.weight'])
assert torch.equal(w2v_state['feature_extractor.conv_layers.3.0.weight'], saved_4ch_w2v_state['feature_extractor.conv_layers.3.0.weight'])


In [ ]:
for key in w2v_state.keys():
    if not torch.equal(w2v_state[key], saved_4ch_w2v_state[key]):
        print("key", key)

In [ ]:
assert torch.equal(w2v_state['feature_extractor.conv_layers.0.0.weight'], saved_4ch_w2v_state['feature_extractor.conv_layers.0.0.weight'])
# should raise a error